In [7]:
import scipy.io as sio

# load data
simDataPath = 'C:/Users/13063/Downloads/windData.mat'
simData = sio.loadmat(simDataPath, squeeze_me=True, struct_as_record=False)
U = simData['U']
Z = simData['Z']
dt = simData['dt']
t = simData['t']
del simData
print("\nThe dimension of U is:", U.shape)



The dimension of U is: (11, 6000)


In [11]:
import numpy as np
import pandas as pd

# 读取风速数据

time = t
wind_speed =U[0]

# 参数定义
g = 9.81  # 重力加速度 (m/s^2)
H = 324.95 # 塔高 (m)
rho = 1.225  # 空气密度 (kg/m^3)
A = 2338.74  # 横截面积 (m^2)，假设值
T1 = 6.38  # 周期 (s)
omega1 = 2 * np.pi / T1  # 自然频率 (rad/s)
mu_s = 1.0  # 形状因子
mu_i = 0.5  # 惯性因子
beta = 1/6  # Newmark-β参数
gamma = 1/2  # Newmark-γ参数
dt = 0.1  # 时间步长 (s)
total_time = 600  # 总时间 (s)
n_steps = int(total_time / dt)

# 质量、刚度和阻尼
m = 2500  # 质量 (kg)
k = m * omega1**2  # 刚度 (N/m)
c = 2 * beta * np.sqrt(k * m)  # 阻尼 (Ns/m)

# 初始化位移、速度和加速度
u = np.zeros(n_steps+1)
v = np.zeros(n_steps+1)
a = np.zeros(n_steps+1)

# 计算平均风速和脉动风速
mean_wind_speed = np.mean(wind_speed)
fluctuating_wind_speed = wind_speed - mean_wind_speed

# 风荷载时间历程
def wind_load(t, mean_wind_speed, fluctuating_wind_speed):
    index = int(t // dt)
    u_i = fluctuating_wind_speed[index]
    return 0.5 * rho * A * (mean_wind_speed**2 +mean_wind_speed*u_i*2) * mu_s

# Newmark-β法
for i in range(1, n_steps):
    # 当前风荷载
    F_i = wind_load(time[i], mean_wind_speed, fluctuating_wind_speed)
    
    # 预测加速度和速度
    a_pred = (F_i - c * v[i] - k * u[i]) / m
    v_pred = v[i] + dt * a_pred
    u_pred = u[i] + dt * v_pred + (0.5 - beta) * dt**2 * a_pred
    
    # 计算有效刚度和质量
    K_eff = m / (beta * dt**2) + c / (gamma * dt) + k
    F_eff = m * (1 / (beta * dt**2)) * u_pred + c * (1 / (gamma * dt)) * v_pred + k * u_pred - F_i
    
    # 求解加速度
    a[i+1] = np.linalg.solve(K_eff, F_eff)
    
    # 更新速度和位移
    v[i+1] = v[i] + dt * (a[i] + a[i+1]) * (1 - gamma)
    u[i+1] = u[i] + dt * (v[i] + v[i+1]) * 0.5 + dt**2 * (a[i] + a[i+1]) * (0.5 - beta / (2 * gamma))

# 计算极值位移
extreme_displacement = np.max(np.abs(u))

print(f"极端位移: {extreme_displacement:.2f} m")

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional